In [ ]:
import cv2, numpy as np, os, radialProfile, glob, pickle
from matplotlib import pyplot as plt
from scipy.interpolate import griddata
from sklearn.svm import SVC

In [ ]:
"""
    So far, there are 3 options:
        - 'deepfake'
        - 'face2face'
        - 'faceswap'
"""

generator = "faceswap"

In [ ]:
epsilon = 500
train_pickle ="data/pickled_data/train_"+generator+"_10k_"+str(epsilon)+".pkl"
test_pickle = "data/pickled_data/test_"+generator+"_2k_"+str(epsilon)+".pkl"

### Load / check features on graph

In [ ]:
# load feature file
pkl_file = open(train_pickle, 'rb')
data = pickle.load(pkl_file)
pkl_file.close()
X = data["data"]
y = data["label"]

In [ ]:
plt.plot(y)

### Check Spectrum
(I'm on the spectrum lmao)

In [ ]:
num = int(X.shape[0]/2)
num_feat = X.shape[1]

psd1D_org_0 = np.zeros((num,num_feat))
psd1D_org_1 = np.zeros((num,num_feat))
psd1D_org_0_mean = np.zeros(num_feat)
psd1D_org_0_std = np.zeros(num_feat)
psd1D_org_1_mean = np.zeros(num_feat)
psd1D_org_1_std = np.zeros(num_feat)

cont_0=0
cont_1=0

# We separate real and fake using the label
for x in range(X.shape[0]):
    if y[x]==0:
        psd1D_org_0[cont_0,:] = X[x,:]
        cont_0+=1
    elif y[x]==1:
        psd1D_org_1[cont_1,:] = X[x,:]
        cont_1+=1

# We compute statistcis
for x in range(num_feat):
    psd1D_org_0_mean[x] = np.mean(psd1D_org_0[:,x])
    psd1D_org_0_std[x]= np.std(psd1D_org_0[:,x])
    psd1D_org_1_mean[x] = np.mean(psd1D_org_1[:,x])
    psd1D_org_1_std[x]= np.std(psd1D_org_1[:,x])
    
# Plot
x = np.arange(0, num_feat, 1)
fig, ax = plt.subplots(figsize=(15, 9))
ax.plot(x, psd1D_org_0_mean, alpha=0.5, color='red', label='Fake', linewidth =2.0)
ax.fill_between(x, psd1D_org_0_mean - psd1D_org_0_std, psd1D_org_0_mean + psd1D_org_0_std, color='red', alpha=0.2)
ax.plot(x, psd1D_org_1_mean, alpha=0.5, color='blue', label='Real', linewidth =2.0)
ax.fill_between(x, psd1D_org_1_mean - psd1D_org_1_std, psd1D_org_1_mean + psd1D_org_1_std, color='blue', alpha=0.2)
plt.tick_params(axis='x', labelsize=20)
plt.tick_params(axis='y', labelsize=20)
ax.legend(loc='best', prop={'size': 20})
plt.xlabel("Spatial Frequency", fontsize=20)
plt.ylabel("Power Spectrum", fontsize=20)

### Training SVM

In [ ]:
#Hyperameters
C_val=6.37
gamma_val = 0.86

#train
pkl_file = open(train_pickle, 'rb')
data = pickle.load(pkl_file)
pkl_file.close()
X = data["data"]
y = data["label"]

svclassifier_r = SVC(C=C_val, kernel='rbf', gamma=gamma_val)
svclassifier_r.fit(X, y)

pickle_svm_file = "data/pickled_classifiers/"+generator+"SVM_ep" + str(epsilon) + "_C" + str(C_val) + "_gam" + str(gamma_val) + ".pkl"
output = open(pickle_svm_file, "wb+")
pickle.dump(svclassifier_r, output)
output.close()

### Testing SVM

In [ ]:
#test
pkl_file = open(test_pickle, 'rb')
data = pickle.load(pkl_file)
pkl_file.close()
X_ = data["data"]
y_ = data["label"]

SVM = svclassifier_r.score(X_, y_)
print("SVM: "+str(SVM))